In [15]:
import numpy as np

import pandas as pd

import bokeh
import bokeh.plotting
from bokeh.plotting import ColumnDataSource
from bokeh.models import LabelSet
from bokeh.models import FuncTickFormatter
import bokeh.io
bokeh.io.output_notebook()

Loading BokehJS ...

In [4]:
df = pd.read_excel('191031_Deuterium_Transfer_Peak_Areas.xlsx')

In [5]:
df.tail()

,Species,WellNumber,Peak ID,Ret Time,Start Tm,End Tm,m/z,Area,Area Percent,Height,Height Percent,A/H,Type,Outlier
43,L,8,D50C24,9.491,9.465,9.515,TIC,107896,2.80,109362,2.50,0.99,L,N
44,L,8,D62C30,12.336,12.320,12.365,TIC,179711,4.66,99836,2.28,1.80,L,N
45,D,8,C18,7.428,7.405,7.450,TIC,3218004,99.46,4022446,99.63,0.80,L,N
46,D,8,D50C24,9.496,9.480,9.510,TIC,3777,0.12,3328,0.08,1.13,L,N
47,D,8,D62C30,12.365,12.320,12.365,TIC,13626,0.42,11795,0.29,1.16,L,N


In [6]:
c18_Areas = np.repeat(df['Area'].loc[df['Peak ID'] == 'C18'],3).reset_index().drop("index", axis=1)
df['Normalized Area'] = np.divide(df['Area'],c18_Areas['Area'])
df['Hydrocarbon amount'] = df['Normalized Area']*25

In [10]:
df['Identifier']=''
for i in range(8):
    curSample = df.loc[df['WellNumber'] == i+1]
    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'U'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSU'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SU'

    if (any((curSample['Species'] == 'S') & (curSample['Type'] == 'L'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LSL'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'S')] = 'SL'


    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'U'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPU'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PU'


    if (any((curSample['Species'] == 'P') & (curSample['Type'] == 'L'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LPL'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'P')] = 'PL'

    if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'U'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDU'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DU'


    if (any((curSample['Species'] == 'D') & (curSample['Type'] == 'L'))):
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'L')] = 'LDL'
        df['Identifier'].loc[(df['WellNumber']==i+1) & (df['Species'] == 'D')] = 'DL'


/Users/TomNaragon/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [20]:
df_C24 = df.loc[df['Peak ID'] == 'D50C24']
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated',
        'LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated',
        'LDU': '','DU': 'Dalotia Control','LDL': '','DL': 'Dalotia Treated'}
np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D50C24',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for _,i in enumerate(np.unique(df_C24['WellNumber'])):

    data = {'identifier': df_C24['Identifier'].loc[(df_C24['WellNumber']==i)].values,
            'hydrocarbon amount': df_C24['Hydrocarbon amount'].loc[(df_C24['WellNumber']==i)].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color='black',
           alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)


p.xgrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)

In [19]:
df_C30 = df.loc[df['Peak ID'] == 'D62C30']
Identifiers = ['LSU','SU','LSL','SL','LPU','PU','LPL','PL','LDU','DU','LDL','DL',]
Labels={'LSU': '','SU': 'Sceptobius Control','LSL': '','SL': 'Sceptobius Treated','LPU': '','PU': 'Platyusa Control','LPL': '','PL': 'Platyusa Treated'}
np.random.seed(666)  

p = bokeh.plotting.figure(plot_width=600,
                          plot_height=600,
                          title='C24',
                          x_range=Identifiers,
                          y_axis_label='ng D62C30',
                          y_axis_type='log')

colors=['#494949','#5F56FF','#494949','#5F56FF','#494949','#C42F2F','#494949','#C42F2F','#494949','#832161','#494949','#832161']

for _,i in enumerate(np.unique(df_C30['WellNumber'])):

    data = {'identifier': df_C30['Identifier'].loc[(df_C30['WellNumber']==i)].values,
            'hydrocarbon amount': df_C30['Hydrocarbon amount'].loc[(df_C30['WellNumber']==i)].values}

    source = ColumnDataSource(data=data)

    offsetVal=(np.random.rand(1)[0]-0.5)*0.5
    p.line(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color='black',
           alpha=0.3)

    p.circle(bokeh.transform.dodge('identifier',  offsetVal,  range=p.x_range),
           'hydrocarbon amount',
           source=source,
           color=bokeh.transform.factor_cmap('identifier',colors,Identifiers),
           alpha=0.6,
           size=7)


p.xgrid.visible = False

# Add custom axis

p.xaxis.formatter = FuncTickFormatter(code="""
var labels = %s;
return labels[tick];
""" %Labels)

bokeh.io.show(p)